In [80]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

## 1. Reading data

In [81]:
data = pd.read_csv("mushroom.csv")
X, y = data.drop('class', axis=1).copy(), data['class'].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
data.head()

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,p,15.26,x,g,o,f,e,NaN,w,16.95,...,s,y,w,u,w,t,g,NaN,d,w
1,p,16.60,x,g,o,f,e,NaN,w,17.99,...,s,y,w,u,w,t,g,NaN,d,u
2,p,14.07,x,g,o,f,e,NaN,w,17.80,...,s,y,w,u,w,t,g,NaN,d,w
3,p,14.17,f,h,e,f,e,NaN,w,15.77,...,s,y,w,u,w,t,p,NaN,d,w
4,p,14.64,x,h,o,f,e,NaN,w,16.53,...,s,y,w,u,w,t,p,NaN,d,w


In [66]:
#по умолчанию describe делается по числовым колонкам
data.describe()

,cap-diameter,stem-height,stem-width
count,61069.000000,61069.000000,61069.000000
mean,6.733854,6.581538,12.149410
std,5.264845,3.370017,10.035955
min,0.380000,0.000000,0.000000
25%,3.480000,4.640000,5.210000
50%,5.860000,5.950000,10.190000
75%,8.540000,7.740000,16.570000
max,62.340000,33.920000,103.910000


In [67]:
#describe по не числовым колонкам
data.describe(exclude=[np.number])

,class,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
count,61069,61069,46949,61069,61069,51185,36006,61069,9531,22945,61069,3177,7413,61069,58598,6354,61069,61069
unique,2,7,11,12,2,7,3,12,5,8,13,1,6,2,8,7,8,4
top,p,x,t,n,f,a,c,w,s,s,w,u,w,f,f,k,d,a
freq,33888,26934,8196,24218,50479,12698,24710,18521,3177,6025,22926,3177,5474,45890,48361,2118,44209,30177


In [78]:
NaN_percent = X.isnull().sum(axis=0) / len(X) * 100 # % NaN
print(NaN_percent)
#баланс классов
print('-'*20)
print(y.describe())
print({k:(y == k).mean() for k in y.unique()})

cap-diameter             0.000000
cap-shape                0.000000
cap-surface             23.121387
cap-color                0.000000
does-bruise-or-bleed     0.000000
gill-attachment         16.184971
gill-spacing            41.040462
gill-color               0.000000
stem-height              0.000000
stem-width               0.000000
stem-root               84.393064
stem-surface            62.427746
stem-color               0.000000
veil-type               94.797688
veil-color              87.861272
has-ring                 0.000000
ring-type                4.046243
spore-print-color       89.595376
habitat                  0.000000
season                   0.000000
dtype: float64
--------------------
count     61069
unique        2
top           p
freq      33888
Name: class, dtype: object
{'p': 0.5549132947976878, 'e': 0.44508670520231214}


* Существует проблема, что есть признаки, у которых  большое количество пропусков, поэтому для некоторых моделей это может стать проблемой

  Так как я не разбираюсь в грибах принимаю решение избавиться от тех признаков, что содержат много пропусков, а там где пропусков мало заполним модой

* Классы выглядят сбалансированными

## OneRule

In [79]:
from sklearn.dummy import DummyClassifier

In [85]:
#OneR с определением лучшей стратегии
strategy_list = ['most_frequent', 'prior', 'stratified', 'uniform']
strategy_score = {}
for strategy in strategy_list:
    OneR = DummyClassifier(strategy=strategy).fit(X_train, y_train)
    score = OneR.score(X_test, y_test) #Return the mean accuracy on the given test data and labels
    strategy_score[strategy] = score
print(strategy_score)

{'most_frequent': 0.5582118879973801, 'prior': 0.5582118879973801, 'stratified': 0.509633753616069, 'uniform': 0.49828066153594236}


In [88]:
X_test[[NaN_percent < 5]]

KeyError: "None of [Index([(True, True, False, True, True, False, False, True, True, True, False, False, True, False, False, True, True, False, True, True)], dtype='object')] are in the [columns]"

In [86]:
#OneR без null признаков
strategy_list = ['most_frequent', 'prior', 'stratified', 'uniform']
strategy_score = {}
for strategy in strategy_list:
    OneR = DummyClassifier(strategy=strategy).fit(X_train[NaN_percent < 5], y_train)
    score = OneR.score(X_test[NaN_percent < 5], y_test) #Return the mean accuracy on the given test data and labels
    strategy_score[strategy] = score
print(strategy_score)

/tmp/ipykernel_3075/1165876386.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  OneR = DummyClassifier(strategy=strategy).fit(X_train[NaN_percent < 5], y_train)


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

## TwoRule